# Convolutional Neural Network

As we did before, we will first load the MNIST dataset.

To use 2d convolutional filters, we reshape the images into 2d.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets('./data', one_hot=True, reshape=False)

print()
print("Image Shape: {}".format(mnist.train.images[0].shape))
print("Label Shape: {}".format(mnist.train.labels[0].shape))
print()
print("Training Set:   {} samples".format(len(mnist.train.images)))
print("Test Set:       {} samples".format(len(mnist.test.images)))

### Building LeNet-5 with Convolution and Pooling operations

![](https://cdnpythonmachinelearning.azureedge.net/wp-content/uploads/2017/09/lenet-5-825x285.png?x64257)

- conv2d: https://www.tensorflow.org/api_docs/python/tf/contrib/layers/conv2d
- max_pool2d: https://www.tensorflow.org/api_docs/python/tf/contrib/layers/max_pool2d

In [ ]:
import tensorflow as tf

In [ ]:
def lenet5(x):
    # Create the model
    # [batch_size, image_width, image_height, channels]
    # convolutional layer
    # input = 28x28x1, output = 24x24x6
    conv1 = tf.contrib.layers.conv2d(
        inputs=x,
        num_outputs=6,
        kernel_size=[5, 5],
        padding='VALID', #SAME
        activation_fn=tf.nn.relu)
    # pooling layer
    # input = 24x24x6, output = 12x12x6
    pool1 = tf.contrib.layers.max_pool2d(
        inputs=conv1,
        kernel_size=[2, 2],
        stride=2,
        padding = 'VALID')
    # convolutional layer
    # input = 12x12x6, output = 6x6x16
    conv2 = tf.contrib.layers.conv2d(
        inputs=pool1,
        num_outputs=16,
        kernel_size=[5, 5],
        padding='VALID',
        activation_fn=tf.nn.relu)
    # pooling layer
    # input = 6x6x16, output = 3x3x16
    pool2 = tf.contrib.layers.max_pool2d(
        inputs=conv2,
        kernel_size=[2, 2],
        stride=2,
        padding = 'VALID')
    # flatten
    # input = 3x3x16, output = 144
    fc1 = tf.contrib.layers.flatten(pool2)
    # fully-connected
    # input = 144, output = 120
    fc2 = tf.contrib.layers.fully_connected(fc1, 120, activation_fn=tf.nn.relu)
    # fully-connected
    # input = 120, output = 84
    fc3 = tf.contrib.layers.fully_connected(fc2, 84, activation_fn=tf.nn.relu)
    # fully-connected
    # input = 84, output = 10
    y = tf.contrib.layers.linear(fc3, 10)
    return y

In [ ]:
# hyperparameters
n_epoch = 10
batch_size = 100
learning_rate = 0.01 #0.1

# define placeholders
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, [None, 10])

# Create the model
y = lenet5(x)

# Define loss and optimizer
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# Define additional tensors for evaluation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Create session and initialize all variables (Here, W and b)
# And then train the model
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    n_batch_per_epoch = mnist.train.labels.shape[0] // batch_size

    for epoch in range(n_epoch):
        for _ in range(n_batch_per_epoch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        print('epoch %d: %.4f' % (epoch, test_accuracy))

## [실습] LeNet-5의 Graph를 브라우저에서 확인하기

In [ ]:
writer = tf.summary.FileWriter('./graphs_lenet', tf.get_default_graph())

In [ ]:
writer.close()

# [참고] Learning rate, Optimizer, Batch size

##  The effects of different learning rates

- With low learning rates the improvements will be linear. 

- With high learning rates they will start to look more exponential. 

- Higher learning rates will decay the loss faster, but they get stuck at worse values of loss (green line). 
  - This is because there is too much "energy" in the optimization and the parameters are bouncing around chaotically, unable to settle in a nice spot in the optimization landscape. 
 
> ***Recommendation:***
> Decay your learning rate over the period of the training. For example, halve the learning rate after a fixed number of epochs, or whenever the validation accuracy tops off.

![](http://cs231n.github.io/assets/nn3/learningrates.jpeg)

##  The effects of different optimizers

There are various algorithms of stochastic gradient descent (SGD)

(https://towardsdatascience.com/types-of-optimization-algorithms-used-in-neural-networks-and-ways-to-optimize-gradient-95ae5d39529f)

- Momentom: SGD의 진동 문제를 완화시킴
- Nesterov accelerated gradient (NAG): minima에 근접했을 때 모멘텀이 너무 커서 지나치는 상황을 방지하기 위해 제안됨
- AdaGrad: 파라미터마다 서로 다른 learning rate를 적용하여 자주 업데이트 되는 파라미터는 나중에 학습이 멈출 수 있음 (decaying learning rate problem)
- AdaDelta: decaying learning rate 문제를 해결하기 위한 AdaGrad의 변형
- Adam: 가장 최근에 나온 알고리즘으로, Adaptive Moment Estimation의 약자. 다른 알고리즘과 비교하여 가장 우수하며 자주 쓰임

> ***The two recommended updates to use***: SGD+Nesterov Momentum or Adam.

### 2d example

![](http://cs231n.github.io/assets/nn3/opt2.gif)

### 3d example

![](http://cs231n.github.io/assets/nn3/opt1.gif)

##  The effects of different batch sizes

Advantages/Disadvantages of small batch size (https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network)

Advantages:

- It requires less memory. Since you train network using less number of samples the overall training procedure requires less memory. It's especially important in case if you are not able to fit dataset in memory.

- Typically networks trains faster with mini-batches. That's because we update weights after each propagation. In our example we've propagated 11 batches (10 of them had 100 samples and 1 had 50 samples) and after each of them we've updated network's parameters. If we used all samples during propagation we would make only 1 update for the network's parameter.

Disadvantages:

- The smaller the batch the less accurate estimate of the gradient. In the figure below you can see that mini-batch (green color) gradient's direction fluctuates compare to the full batch (blue color).

![](https://i.stack.imgur.com/lU3sx.png)


### References

- http://cs231n.github.io/neural-networks-3/